In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

# Define image size
Image_size = [244, 244]

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = train_datagen.flow_from_directory(
    '/kaggle/input/fakereal-logo-detection-dataset/genLogoOutput',
    target_size=(224, 224),
    batch_size=15,
    class_mode='categorical'
)

test_data = train_datagen.flow_from_directory(
    '/kaggle/input/fakereal-logo-detection-dataset/output',
    target_size=(224, 224),
    batch_size=15,
    class_mode='categorical'
)

# Build VGG19 model with pre-trained weights
base_model = VGG19(input_shape=Image_size + [3], weights='imagenet', include_top=False)

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Create your own model
x = Flatten()(base_model.output)
output = Dense(63, activation='softmax')(x)
VGG19_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
VGG19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
VGG19_model.summary()

# Fit the model
VGG19_model.fit(train_data, epochs=10, validation_data=test_data)


VGG19_model.save('fakelogo.h5')

import numpy as np

from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_path = '/kaggle/input/fakereal-logo-detection-dataset/output/Google/000003.jpg'
img = load_img(img_path, target_size=(244, 244))
x = img_to_array(img)
x = preprocess_input(x)
preds = VGG19_model.predict(np.array([x]))  # Use parentheses here, not square brackets

class_names = [
    'Adidas', 'Amazon', 'Android', 'Apple', 'Ariel', 'BMW', 'Bic', 'Burger King',
    'Cadbury', 'Chevrolet', 'Chrome', 'Coca Cola', 'Cowbell', 'Dominos', 'Fila',
    'Gillette', 'Google', 'Goya oil', 'Guinness', 'Heinz', 'Honda', 'Hp', 'Huawei',
    'Instagram', 'Kfc', 'Krisspy Kreme', 'Lays', 'Levis', 'Lg', 'Lipton', 'Mars', 'Marvel', 'McDonald',
    'Mercedes Benz', 'Microsoft', 'MnM', 'Mtn', 'Mtn dew', 'NASA', 'Nescafe', 'Nestle', 'Nestle milo',
    'Netflix', 'Nike', 'Nutella', 'Oral b', 'Oreo', 'Pay pal', 'Peak milk', 'Pepsi', 'PlayStation',
    'Pringles', 'Puma', 'Reebok', 'Rolex', 'Samsung', 'Sprite', 'Starbucks', 'Tesla', 'Tiktok',
    'Twitter', 'YouTube', 'Zara'
]

predicted_class = class_names[np.argmax(preds)]
print(predicted_class)
threshold = 0.5  # Set your threshold here

# Assuming preds is a 2D array with shape (1, num_classes)
predicted_class_index = np.argmax(preds)
predicted_probability = preds[0, predicted_class_index]

if predicted_probability >= threshold:
    predicted_class = class_names[predicted_class_index]
    print(f"Predicted class: {predicted_class} with probability: {predicted_probability}")
else:
    print("Prediction below threshold.")